In [1]:
#Import Required Dependencies

from flask import Flask, jsonify
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session


In [2]:
# Initialize Flask app
app = Flask(__name__)

In [3]:
import os

file_path = "/Users/jeff.gary/Desktop/Starter_Code 2/Resources/hawaii.sqlite"  # Replace with the actual path
print(os.path.exists(file_path))





True


In [4]:
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

file_path = "/Users/jeff.gary/Desktop/Starter_Code 2/Resources/hawaii.sqlite"  # Replace with the actual path
print(os.path.exists(file_path))


if os.path.exists(file_path):
    # Attempt to create engine
    try:
        engine = create_engine(f"sqlite:///{file_path}")
        Base = automap_base()
        Base.prepare(engine)
        Measurement = Base.classes.measurement
        Station = Base.classes.station
        session = Session(engine)
        print("Database connection successful!")
    except Exception as e:
        print(f"Error connecting to database: {e}")
else:
    print("File not found. Please verify the path.")


True
Database connection successful!


In [5]:
# Define API Routes

@app.route("/")
def welcome():
    """List all available API routes."""
    return (
        f"Welcome to the Climate App API!<br/><br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end><br/>"
    )


In [6]:
# Retrieve last 12 months of precipitation data

@app.route("/api/v1.0/precipitation")
def precipitation():
    """Return the precipitation data for the last 12 months."""
    # Find the most recent date in the dataset
    latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    one_year_ago = dt.datetime.strptime(latest_date, "%Y-%m-%d") - dt.timedelta(days=365)

    # Query precipitation data for the last year
    precipitation_data = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= one_year_ago).all()

    # Convert the query results to a dictionary
    precipitation_dict = {date: prcp for date, prcp in precipitation_data}
    return jsonify(precipitation_dict)


In [7]:
# Return a JSON list of all stations

@app.route("/api/v1.0/stations")
def stations():
    """Return a list of all stations."""
    results = session.query(Station.name).all()
    stations = [station[0] for station in results]
    return jsonify(stations)


In [8]:
# Return temperature observations for the most active station for the last 12 months

@app.route("/api/v1.0/tobs")
def tobs():
    """Return temperature observations for the most active station for the last year."""
    # Find the most active station
    most_active_station = session.query(Measurement.station).\
        group_by(Measurement.station).\
        order_by(func.count(Measurement.station).desc()).first()[0]

    # Find the most recent date and calculate the date one year ago
    latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    one_year_ago = dt.datetime.strptime(latest_date, "%Y-%m-%d") - dt.timedelta(days=365)

    # Query temperature observations for the last year for the most active station
    results = session.query(Measurement.date, Measurement.tobs).\
        filter(Measurement.station == most_active_station).\
        filter(Measurement.date >= one_year_ago).all()

    # Convert the results to a list of dictionaries
    temperature_list = [{"date": date, "temperature": tobs} for date, tobs in results]
    return jsonify(temperature_list)


In [9]:
# Return min, avg, and max temperatures for a specified start date

@app.route("/api/v1.0/<start>")
def start_date(start):
    """Return temperature stats (min, avg, max) for dates >= start date."""
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    results = session.query(*sel).filter(Measurement.date >= start).all()

    # Convert the results into a dictionary
    temperature_stats = {
        "TMIN": results[0][0],
        "TAVG": results[0][1],
        "TMAX": results[0][2]
    }
    return jsonify(temperature_stats)


In [10]:
# Return min, avg, and max temperatures for a specified start and end date

@app.route("/api/v1.0/<start>/<end>")
def start_end_date(start, end):
    """Return temperature stats (min, avg, max) for dates between start and end."""
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    results = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()

    # Convert the results into a dictionary
    temperature_stats = {
        "TMIN": results[0][0],
        "TAVG": results[0][1],
        "TMAX": results[0][2]
    }
    return jsonify(temperature_stats)



In [11]:
# Run app to test routes

if __name__ == "__main__":
    app.run(debug=True, port=5002)  # Use a different port, e.g., 5002



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jeff.gary/Library/Python/3.13/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
    ~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/jeff.gary/Library/Python/3.13/lib/python/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
    ~~~~~~~~~~~~~~^^^^^^
  File "/Users/jeff.gary/Library/Python/3.13/lib/python/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/Users/jeff.gary/Library/Python/3.13/lib/python/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
    ~~~~~~~~~~~~~~~~~^^
  File "/Users/jeff.gary/Library/Python/3.13/lib/python/site-packages/ipykernel/kernelapp.

SystemExit: 1

/Users/jeff.gary/Library/Python/3.13/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
